In [11]:
from pathlib import Path
import pandas as pd

DIRECTORIO_DATOS = Path.cwd()

telemetria = pd.read_csv(DIRECTORIO_DATOS / "PdM_telemetry.csv", parse_dates=["datetime"])
errores = pd.read_csv(DIRECTORIO_DATOS / "PdM_errors.csv", parse_dates=["datetime"])
fallas = pd.read_csv(DIRECTORIO_DATOS / "PdM_failures.csv", parse_dates=["datetime"])
mantenciones = pd.read_csv(DIRECTORIO_DATOS / "PdM_maint.csv", parse_dates=["datetime"])
maquinas = pd.read_csv(DIRECTORIO_DATOS / "PdM_machines.csv")

for nombre, df in [
    ("telemetria", telemetria),
    ("errores", errores),
    ("fallas", fallas),
    ("mantenciones", mantenciones),
    ("maquinas", maquinas)
]:
    print(f"{nombre}: {df.shape}")

telemetria: (876100, 6)
errores: (3919, 3)
fallas: (761, 3)
mantenciones: (3286, 3)
maquinas: (100, 3)


In [12]:
def revisar_duplicados(df, columnas, nombre):
    cantidad = df.duplicated(columnas).sum()
    print(f"[{nombre}] duplicados en {columnas}: {cantidad}")

revisar_duplicados(telemetria, ["machineID", "datetime"], "telemetria")
revisar_duplicados(errores, ["machineID", "datetime", "errorID"], "errores")
revisar_duplicados(mantenciones, ["machineID", "datetime", "comp"], "mantenciones")
revisar_duplicados(fallas, ["machineID", "datetime", "failure"], "fallas")

print("\nRango fechas:")
print("telemetria:", telemetria["datetime"].min(), "→", telemetria["datetime"].max())
print("fallas     :", fallas["datetime"].min(), "→", fallas["datetime"].max())
print("mantenciones:", mantenciones["datetime"].min(), "→", mantenciones["datetime"].max())


[telemetria] duplicados en ['machineID', 'datetime']: 0
[errores] duplicados en ['machineID', 'datetime', 'errorID']: 0
[mantenciones] duplicados en ['machineID', 'datetime', 'comp']: 0
[fallas] duplicados en ['machineID', 'datetime', 'failure']: 0

Rango fechas:
telemetria: 2015-01-01 06:00:00 → 2016-01-01 06:00:00
fallas     : 2015-01-02 03:00:00 → 2015-12-31 06:00:00
mantenciones: 2014-06-01 06:00:00 → 2016-01-01 06:00:00


In [13]:
import numpy as np

telemetria["fecha"] = telemetria["datetime"].dt.floor("D")

features_diarios = (
    telemetria
    .groupby(["machineID", "fecha"])
    .agg(["mean", "std", "min", "max"])
)

features_diarios.columns = ["_".join(col).strip() for col in features_diarios.columns]
features_diarios = features_diarios.reset_index()

print("features_diarios:", features_diarios.shape)
features_diarios.head()


features_diarios: (36600, 22)


,machineID,fecha,datetime_mean,datetime_std,datetime_min,datetime_max,volt_mean,volt_std,volt_min,volt_max,...,rotate_min,rotate_max,pressure_mean,pressure_std,pressure_min,pressure_max,vibration_mean,vibration_std,vibration_min,vibration_max
0,1,2015-01-01,2015-01-01 14:30:00,0 days 05:20:18.740853656,2015-01-01 06:00:00,2015-01-01 23:00:00,167.576533,9.300337,151.335682,182.739113,...,346.149335,527.349825,98.522345,10.588562,75.237905,113.077935,40.049623,5.739395,25.990511,51.021486
1,1,2015-01-02,2015-01-02 11:30:00,0 days 07:04:15.844122715,2015-01-02 00:00:00,2015-01-02 23:00:00,169.795758,15.742155,140.776309,200.872430,...,384.645962,519.452812,98.454608,11.679314,78.880780,127.014498,39.271645,5.579524,29.527665,52.355876
2,1,2015-01-03,2015-01-03 11:30:00,0 days 07:04:15.844122715,2015-01-03 00:00:00,2015-01-03 23:00:00,171.862244,11.182853,154.199258,194.942847,...,374.127148,568.972310,97.998233,8.884765,85.246610,116.008404,48.074091,8.194927,32.323616,66.764515
3,1,2015-01-04,2015-01-04 11:30:00,0 days 07:04:15.844122715,2015-01-04 00:00:00,2015-01-04 23:00:00,174.792428,19.224657,129.016707,215.656488,...,365.213804,517.348533,101.452266,10.807630,82.400818,118.853452,52.190268,5.081258,41.674887,62.464103
4,1,2015-01-05,2015-01-05 11:30:00,0 days 07:04:15.844122715,2015-01-05 00:00:00,2015-01-05 23:00:00,171.018408,17.900560,127.163620,202.520488,...,376.719605,575.505189,102.363114,10.672868,78.721961,126.464580,43.330311,8.087134,33.156011,59.577251


In [15]:
fallas["fecha"] = fallas["datetime"].dt.floor("D")

fallas_diarias = (
    fallas
    .groupby(["machineID", "fecha"])
    .size()
    .reset_index(name="cantidad_fallas")
)

fallas_diarias["evento"] = (fallas_diarias["cantidad_fallas"] > 0).astype(int)

print("fallas_diarias:", fallas_diarias.shape)
fallas_diarias.head()

fallas_diarias: (718, 4)


,machineID,fecha,cantidad_fallas,evento
0,1,2015-01-05,1,1
1,1,2015-03-06,1,1
2,1,2015-04-20,1,1
3,1,2015-06-19,1,1
4,1,2015-09-02,1,1


In [16]:
dataset_survival = features_diarios.merge(
    fallas_diarias[["machineID", "fecha", "evento"]],
    on=["machineID", "fecha"],
    how="left"
)

dataset_survival["evento"] = dataset_survival["evento"].fillna(0).astype(int)

dataset_survival = dataset_survival.merge(maquinas, on="machineID", how="left")

dataset_survival = dataset_survival.sort_values(["machineID", "fecha"]).reset_index(drop=True)

print("dataset_survival:", dataset_survival.shape)
dataset_survival.head()


dataset_survival: (36600, 25)


,machineID,fecha,datetime_mean,datetime_std,datetime_min,datetime_max,volt_mean,volt_std,volt_min,volt_max,...,pressure_std,pressure_min,pressure_max,vibration_mean,vibration_std,vibration_min,vibration_max,evento,model,age
0,1,2015-01-01,2015-01-01 14:30:00,0 days 05:20:18.740853656,2015-01-01 06:00:00,2015-01-01 23:00:00,167.576533,9.300337,151.335682,182.739113,...,10.588562,75.237905,113.077935,40.049623,5.739395,25.990511,51.021486,0,model3,18
1,1,2015-01-02,2015-01-02 11:30:00,0 days 07:04:15.844122715,2015-01-02 00:00:00,2015-01-02 23:00:00,169.795758,15.742155,140.776309,200.872430,...,11.679314,78.880780,127.014498,39.271645,5.579524,29.527665,52.355876,0,model3,18
2,1,2015-01-03,2015-01-03 11:30:00,0 days 07:04:15.844122715,2015-01-03 00:00:00,2015-01-03 23:00:00,171.862244,11.182853,154.199258,194.942847,...,8.884765,85.246610,116.008404,48.074091,8.194927,32.323616,66.764515,0,model3,18
3,1,2015-01-04,2015-01-04 11:30:00,0 days 07:04:15.844122715,2015-01-04 00:00:00,2015-01-04 23:00:00,174.792428,19.224657,129.016707,215.656488,...,10.807630,82.400818,118.853452,52.190268,5.081258,41.674887,62.464103,0,model3,18
4,1,2015-01-05,2015-01-05 11:30:00,0 days 07:04:15.844122715,2015-01-05 00:00:00,2015-01-05 23:00:00,171.018408,17.900560,127.163620,202.520488,...,10.672868,78.721961,126.464580,43.330311,8.087134,33.156011,59.577251,1,model3,18


In [18]:
fechas_fallas = (
    fallas[["machineID", "fecha"]]
    .drop_duplicates()
    .sort_values(["machineID", "fecha"])
)

dataset_survival["proxima_falla"] = pd.NaT

for maquina_id, grupo in dataset_survival.groupby("machineID", sort=False):
    fechas = fechas_fallas.loc[fechas_fallas["machineID"] == maquina_id, "fecha"].values
    fechas_obs = grupo["fecha"].values

    # Si esta máquina no tiene fallas registradas (por si acaso)
    if len(fechas) == 0:
        dataset_survival.loc[grupo.index, "proxima_falla"] = pd.NaT
        continue

    indices = np.searchsorted(fechas, fechas_obs, side="left")

    # Creamos vector resultado lleno con NaT
    prox = np.full(shape=len(indices), fill_value=np.datetime64("NaT"), dtype="datetime64[ns]")

    # Solo donde el índice cae dentro del rango, asignamos la fecha
    mascara_valida = indices < len(fechas)
    prox[mascara_valida] = fechas[indices[mascara_valida]]

    dataset_survival.loc[grupo.index, "proxima_falla"] = prox

dataset_survival["dias_hasta_falla"] = (dataset_survival["proxima_falla"] - dataset_survival["fecha"]).dt.days
dataset_survival["censurado"] = dataset_survival["proxima_falla"].isna().astype(int)

dataset_survival["dias_hasta_falla"] = dataset_survival["dias_hasta_falla"].fillna(-1).astype(int)

dataset_survival[["machineID", "fecha", "evento", "proxima_falla", "dias_hasta_falla", "censurado"]].head(20)


,machineID,fecha,evento,proxima_falla,dias_hasta_falla,censurado
0,1,2015-01-01,0,2015-01-05,4,0
1,1,2015-01-02,0,2015-01-05,3,0
2,1,2015-01-03,0,2015-01-05,2,0
3,1,2015-01-04,0,2015-01-05,1,0
4,1,2015-01-05,1,2015-01-05,0,0
5,1,2015-01-06,0,2015-03-06,59,0
6,1,2015-01-07,0,2015-03-06,58,0
7,1,2015-01-08,0,2015-03-06,57,0
8,1,2015-01-09,0,2015-03-06,56,0
9,1,2015-01-10,0,2015-03-06,55,0


In [20]:
columnas_clave = {"machineID","fecha","evento","dias_hasta_falla","censurado"}
print("Columnas faltantes:", columnas_clave - set(dataset_survival.columns))
print("Filas:", len(dataset_survival), "Máquinas:", dataset_survival["machineID"].nunique())
print("Rango fechas:", dataset_survival["fecha"].min(), "→", dataset_survival["fecha"].max())
print("% censurados:", dataset_survival["censurado"].mean())
dataset_survival[["machineID","fecha","evento","dias_hasta_falla","censurado"]].head()

Columnas faltantes: set()
Filas: 36600 Máquinas: 100
Rango fechas: 2015-01-01 00:00:00 → 2016-01-01 00:00:00
% censurados: 0.1375136612021858


,machineID,fecha,evento,dias_hasta_falla,censurado
0,1,2015-01-01,0,4,0
1,1,2015-01-02,0,3,0
2,1,2015-01-03,0,2,0
3,1,2015-01-04,0,1,0
4,1,2015-01-05,1,0,0


In [19]:
DIRECTORIO_SALIDA = Path.cwd() / "data" / "processed" / "azure_pm"
DIRECTORIO_SALIDA.mkdir(parents=True, exist_ok=True)

ruta_salida = DIRECTORIO_SALIDA / "dataset_survival_diario.parquet"

# requiere pyarrow: pip install pyarrow
dataset_survival.to_parquet(ruta_salida, index=False)

print("Guardado en:", ruta_salida.resolve())


Guardado en: C:\Users\sebas\OneDrive\Desktop\Proyecto Chatbot\Mantenimiento Industrial\data\processed\azure_pm\dataset_survival_diario.parquet
